In [ ]:
from verification_net import VerificationNet
from stacked_mnist import StackedMNIST, DataMode

import matplotlib.pyplot as plt

import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import MNIST

mode = DataMode.MONO 

trainset = StackedMNIST(train=True, mode=mode)
testset = StackedMNIST(train=False, mode=mode)

batch_size = 25000

In [ ]:
net = VerificationNet(force_learn=False, file_name='models/verification_model')

net.train(trainset=trainset, valset=testset, epochs=15, batch_size=1024)

plt.figure()
plt.plot(net.losses, label="train")
plt.plot(net.val_loss, label="validation")
plt.legend()
plt.plot()

train_set = DataLoader(trainset, shuffle=True, batch_size=batch_size)

for idx, (imgs, targets, labels) in enumerate(train_set, 0):
    labels = labels.to('cpu').detach().numpy()
    break

cov = net.check_class_coverage(data=imgs, tolerance=.8)
pred, acc = net.check_predictability(data=imgs, correct_labels=labels, tolerance=.8)
print(f"Coverage: {100*cov:.2f}%")
print(f"Predictability: {100*pred:.2f}%")
print(f"Accuracy: {100 * acc:.2f}%")

from torchvision.transforms import ToPILImage

img = ToPILImage()(imgs[0])
img